# Vinmonopolet - Scraping for gold

In [1]:
#Importing all necessary packages
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import re
import time
from difflib import SequenceMatcher


In [2]:
#Opening Chrome in a new window
driver = webdriver.Chrome()

#Going to the intended url
driver.get("https://www.vinmonopolet.no/search/?q=:relevance:price:750:price:1000:price:5000&searchType=product&currentPage=0")

In [ ]:
#Creating a dictionary to store all wine values
wines = {
    "Name" : [],
    "Year" : [],
    "Price" : [],
    "Cl" : [],
    "Country" : [],
    "Category" : [],
    "Region" : [],
    "ID" : []
}

#Setting stop value for the while-loop
stop = True
tmp = 0 #Need variable to do checks on first/last page

#Setting default values in case I find no information
region_nf = "not found"
year_nf = "Non-Vintage"

#While-loop, iterating all articles on Vinmonopolet
while stop:
    
    #Finding all items on the current page
    wine_list = driver.find_elements(By.XPATH,'//li[contains(@class,"product-item")]')
    tmp += len(list(wine_list))
    
    #Iterating through the wines
    for i in range(len(wine_list)):
        
        #Getting the name stored in 'div'-tag
        name = driver.find_elements(By.XPATH, '//div[contains(@class, "product__name")]')[i].text
        
        #Getting year, which is (if present) the last part of the name
        year = name.split(" ")
        
        #Iterating through the word to find the year
        for word in year:
            if word.isnumeric():
                if len(word) == 4:
                    year = word
                    #I do not break the loop here in case we have more years in the name (it does excist)
            else:
                year = year_nf
        
        #Setting the category stored in a 'div'-tag
        category = driver.find_elements(By.XPATH, '//div[contains(@class, "product__category-name")]')[i].text.capitalize()
        
        #Getting both country and region, which is both stored in a string (in a 'div'-tag), splitting by comma-delimiter
        country_region = driver.find_elements(By.XPATH, '//div[contains(@class, "product__district")]')[i].text.split(",")
        
        #Setting region-value in case we don't find anything
        region = ""
        
        #Checking whether we have both country and region
        if len(country_region)==1:
            country = country_region[0]
        else:
            country = country_region[0]
            region = country_region[1]
        
        #Getting price, and doing necessary formatting. 
        price = driver.find_elements(By.XPATH, '//span[contains(@class, "product__price")]')[i].text
        price = price.replace(",", ".").replace(u"\xa0", u"").replace(u"\xf4", u"o").replace(u"\xf3",u"o").replace("Kr", "")
        price = float(price.replace(u" ", u"").strip())/11.5 #We divide by 11.5 as that was the NOKGBP at the time of writing
        price = "{:.2f}".format(price)
        
        #Getting the amount of litres in the bottle
        cl = driver.find_elements(By.XPATH, '//span[contains(@class, "product__amount")]')[i].text
        
        #Getting the ordering ID to make it easier for us to access the right wine at a later point (if we were to move on with this idea)
        _id = driver.find_elements(By.XPATH, '//div[contains(@class, "product__code")]')[i].text
        
        #Adding all the variables to the dictionary
        wines["Name"].append(name)
        wines["Cl"].append(cl)
        wines["Country"].append(country)
        wines["Category"].append(category)
        wines["ID"].append(_id)
        wines["Price"].append(price)
        wines["Year"].append(year)
        
        #Adding region if it is not empty, else we add the default value of "not found"
        if region == "":
            wines["Region"].append(region_nf)
        else:
            wines["Region"].append(region)
        
    #Checking whether there is a next_arrow
    next_arrows = driver.find_elements(By.XPATH,'//button[contains(@class,"pagination-button")]')
    if len(next_arrows) > 0:  
        #If there is only one arrow, it is either the first or last page
        if len(next_arrows)==1:
            #Doing this check to check whether it is the first page or not, if not, we are at the last page
            if tmp < 25:
                next_arrow = next_arrows[0]
            else:
                break
        else:
            next_arrow = next_arrows[1]
        
        #Clicking the next_arrow
        if next_arrow.is_enabled():
            next_arrow.click()
            time.sleep(3) # Need to wait for some time to load the webpage
        else:
            stop = False
    else:
        stop = False
        

#Creating a DataFrame of the dataset
df = pd.DataFrame(wines)

#Translation

#Reading the self-made dictionary especially for this assignment
translations = pd.read_csv("translations.csv")

#Dropping any NA's that could be included
translations = translations.dropna()

#Creating a list of all words that need to be translated
translated_words = list(translations.loc[:,"Norwegian"])

#Formatting the strings in translated_words
translated_words.loc[:, "Norwegian"] = translated_words.loc[:, "Norwegian"].str.strip()

#List of all the translation words to do the translation in the loop
translation = list(translations.loc[:,"English"])

#For-loop over all wines from Vinmonopolet
for row in range(df.shape[0]):
    
    #Doing IF-checks on the columns that need translation
    #Category
    if df.loc[row, "Category"] in translated_words:
        df.loc[row, "Category"] = translation[translated_words.index(df.loc[row, "Category"])]
    
    #Country
    if df.loc[row, "Country"] in translated_words:
        df.loc[row, "Country"] = translation[translated_words.index(df.loc[row, "Country"])]
    
    #Region
    if df.loc[row, "Region"] in translated_words:
        df.loc[row, "Region"] = translation[translated_words.index(df.loc[row, "Region"])]

In [10]:
#Storing the dataset to a csv-file
df.to_csv("wines_vinmonopolet.csv")


In [11]:
#Just to load this if I have ran the algorithm earlier as it take a lot of time to run the algorithm
df = pd.read_csv("wines_vinmonopolet.csv")

In [12]:
#Read the dataset from Vivino (Stored as an excel-file)
df_vivino = pd.read_excel("wine_scrape_clean_result.xlsx")

In [ ]:

#Creating a dictionary to store values
arbitrage_wines = {
    "name" : [],
    "vpolet_price" : [],
    "vivino_price" : [],
    "region" : [],
    "country" : [],
    "ratings" : []
}

#Double for-loop to find all of the matching wines (with 85% accuracy)

for i in range(df.shape[0]):
    for j in range(df_vivino.shape[0]):
        
        #Checking if the algorithm matches by 85%
        if float(SequenceMatcher(None, df.loc[i, "Name"].lower(), (df_vivino.loc[j, "WineBrand"].lower() + " " +  df_vivino.loc[j, "WineName"].lower())).ratio())>0.85:
            
            #Formatting the wine names to check if there is a year in the end (we need matching years as year is treated as a string)

            wine1 = df.loc[i, "Name"].lower()
            wine2 = df_vivino.loc[j, "WineBrand"].lower() + " " +  df_vivino.loc[j, "WineName"].lower()
            wine1 = wine1.split()
            wine2 = wine2.split()
            
            #Year always comes last (if there is a number), so we check the last string in each wine
            if wine1[len(wine1)-1] == wine2[len(wine2)-1]:
                
                #Checking whether there is an arbitrage opportunity
                if float(df.loc[i, "Price"])<float(df_vivino.loc[j, "WinePrice"]):
                    
                    #If there is an arbitrage opportunity, we add the wine to a new dataframe
                    arbitrage_wines["name"].append(df.loc[i, "Name"])
                    arbitrage_wines["vpolet_price"].append(df.loc[i, "Price"])
                    arbitrage_wines["region"].append(df.loc[i, "Region"])
                    arbitrage_wines["country"].append(df.loc[i, "Country"])
                    arbitrage_wines["vivino_price"].append(df_vivino.loc[i, "WinePrice"])
                    arbitrage_wines["ratings"].append(df_vivino.loc[i, "WineRating"])
                    print("\n--------------------\n")
                    
                    break

#Storing all arbitrage opportunities in a dataframe
arbitrage_wines_df = pd.DataFrame.from_dict(arbitrage_wines)

#Create a csv-file with the dataset
arbitrage_wines_df.to_csv("arbitrage_wines.csv")